In [1]:
# Initialization

import math
from datetime import datetime
import openpyxl as opxl
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import model as ty
import proc as pr
import matplotlib.ticker as mtick
import sklearn.metrics

import scipy.constants as scc

import sys
sys.path.append('../RD-systems-and-test-benches')

import thermal.SK_standard as nm

import seaborn as sns

import utils.plot_functions as RD_pfun
import repo_config as rconfig

from IPython.core.display import HTML

import heat_transfer as bht

import fluids as fds
import ht 

import general as gen

import os

import scipy.integrate as integrate
import scipy.optimize as sco

import networkx as nx

import plotly.graph_objects as go

import plot_functions_here as pfun

import plotly.io as pio

from itertools import product
from itertools import chain

from tqdm.notebook import tqdm

import copy

In [2]:
folder_path = r'G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\PVT-perf-1Dmodel'
geometry_path = folder_path + '\\' + 'Panels\\Panel_V4.1I.xlsx'
hypotheses_path = folder_path + '\\' + 'Hypotheses\\Model_hypotheses.xlsx'
condi_path = folder_path + '\\' + 'Steady-state conditions inputs\\100424-tests-conditions-GMI.xlsx'
exp_path = folder_path + '\\' + 'Experiments\\Experiments-GMI.xlsx'

panelSpecs = pr.import_geometry(geometry_path)
hyp = pr.create_dict_from_excel(hypotheses_path,'Main')

# steadyStateConditions_df = pd.read_excel(condi_path,header=2) ## Les deux première lignes ne sont pas à prendre en compte
# stepConditions = steadyStateConditions_df.to_dict(orient='records')

lambd_ins 0.018
lambd_ins 0.0
lambd_ins 0.018
lambd_ins 0.0


In [3]:
recap = pd.read_excel(r'G:\Drive partagés\Cercle Hard\R&D\Thermal performance tests PVT\Recap_RD_TUV_SSI.xlsx')
recap = recap.drop(index=0)
recap.dropna(how='all', inplace=True)

groups = {key: group for key, group in recap.groupby('Design')}

In [4]:
steadyStateConditions_df = groups['V4.1I'].loc[groups['V4.1I']['Type'] == 'SK'].copy()
pr.convert_to_K(steadyStateConditions_df)
pr.adjust_steadyStateConditions(steadyStateConditions_df, hyp)
steadyStateConditions_df.reset_index(drop=True, inplace=True)
steadyStateConditions = steadyStateConditions_df.to_dict(orient='records')

In [5]:
# df_res, list_res = ty.simu_steadyStateConditions(panelSpecs,hyp,steadyStateConditions_df)
# df_one, res = ty.simu_one_steady_state_all_he(panelSpecs, steadyStateConditions[0], hyp)

In [6]:
mapping_parameters_bounds = {
    'panelSpecs': {
        'pv' : {
                'tau_g' : (0.8, 0.95),
                # 'alpha_g' : (0.02,0.08),
                # 'alpha_PV' : (0.8, 0.95),
                # 'eps_PV' : (0.85, 0.95),
                # 'eps_g' : (0.85, 0.95),
        },
        'meander' : {
                # 'l_c' : (1.5e-3,3e-3),
                'lambd_air' : (0.01e-3,1.5e-3),
        }
    },
    'hyp': {
        'coeff_h_top_forced_range0' : (0.5, 2.5),
        'coeff_h_top_forced_range1' : (0.5, 2.5),
        'coeff_h_top_forced_range2' : (0.5, 2.5),
    },
}

# Function to recursively flatten the parameter bounds
def flatten_parameters_bounds(d, parent_keys=[]):
    items = []
    for k, v in d.items():
        if isinstance(v, dict):
            items.extend(flatten_parameters_bounds(v, parent_keys + [k]))
        else:
            items.append((parent_keys + [k], v))
    return items

# Flatten the parameter bounds and extract parameter paths and bounds
flattened_parameters = flatten_parameters_bounds(mapping_parameters_bounds)
parameter_paths = [param[0] for param in flattened_parameters]
bounds = [param[1] for param in flattened_parameters]

mapping_parameters = {tuple(flattened_parameters[j][0]): j for j in range(len(flattened_parameters))}

def update_panelSpecs_and_hyp(params, panelSpecs, hyp,mapping_parameters_bounds):

    # Update panelSpecs
    for key in mapping_parameters_bounds['panelSpecs']:

        if key in list(set(panelSpecs['decomp'].values()))+['pv']:

            zones = list(panelSpecs['decomp'].keys()) if key == 'pv' else [_ for _, value in panelSpecs['decomp'].items() if value == key]

            for z in zones:
                for subkey in mapping_parameters_bounds['panelSpecs'][key]:

                    panelSpecs[z][subkey] = params[mapping_parameters[('panelSpecs', key, subkey)]]
        else:
            panelSpecs[key] = params[mapping_parameters[('panelSpecs', key)]]
    
    # Update hyp
    for key in mapping_parameters_bounds['hyp']:
        hyp[key] = params[mapping_parameters[('hyp', key)]]

def simulation_function(params, panelSpecs, hyp, steadyStateConditions_df, mapping_parameters_bounds):
    
    update_panelSpecs_and_hyp(params, panelSpecs, hyp, mapping_parameters_bounds)

    # Call the simulation function
    df_res, list_res = ty.simu_steadyStateConditions(panelSpecs, hyp, steadyStateConditions_df)
    return df_res

def objective_function(params, panelSpecs, hyp, steadyStateConditions_df, experimental_df_res, mapping_parameters_bounds):
    df_res = simulation_function(params, panelSpecs, hyp, steadyStateConditions_df, mapping_parameters_bounds)

    return np.sqrt(((df_res['Qdot_tube_fluid'] - experimental_df_res['Qdot']) ** 2).mean())


In [7]:
panelSpecs_copy = copy.deepcopy(panelSpecs)
hyp_copy = copy.deepcopy(hyp)

# Number of runs with different initial guesses
num_runs = 5

# Store the results
results = []

# Generate random initial guesses within the bounds
for run in tqdm(range(num_runs), desc="Optimizations"):
    # Random initial guess within bounds
    initial_guess = [np.random.uniform(low=b[0], high=b[1]) for b in bounds]

    # Perform the optimization
    result = sco.minimize(
        objective_function,
        x0=initial_guess,
        args=(panelSpecs_copy, hyp_copy, steadyStateConditions_df, steadyStateConditions_df, mapping_parameters_bounds),
        bounds=bounds,
        method='L-BFGS-B'
    )

    # Store the result
    results.append(result)

params_minima = [[(param[0], result['x'][i]) for i, param in enumerate(flattened_parameters)] for result in results]

df_res_minima = []
rmse_minima = []

for params_dict in params_minima:
    df_res = simulation_function([params[1] for params in params_dict], panelSpecs_copy, hyp_copy, steadyStateConditions_df, mapping_parameters_bounds)
    df_res_minima.append(df_res)
    rmse_minima.append(np.sqrt(((df_res['Qdot_tube_fluid'] - steadyStateConditions_df['Qdot']) ** 2).mean()))

Optimizations:   0%|          | 0/5 [00:00<?, ?it/s]

In [8]:
params_minima

[[(['panelSpecs', 'pv', 'tau_g'], 0.8789366419156923),
  (['panelSpecs', 'meander', 'lambd_air'], 0.0009151227181092382),
  (['hyp', 'coeff_h_top_forced_range0'], 2.145223763192959),
  (['hyp', 'coeff_h_top_forced_range1'], 2.1503873907673334),
  (['hyp', 'coeff_h_top_forced_range2'], 0.5905720130194461)],
 [(['panelSpecs', 'pv', 'tau_g'], 0.8789367274993654),
  (['panelSpecs', 'meander', 'lambd_air'], 0.00047341316220261907),
  (['hyp', 'coeff_h_top_forced_range0'], 1.8597697151521944),
  (['hyp', 'coeff_h_top_forced_range1'], 0.8946218799421279),
  (['hyp', 'coeff_h_top_forced_range2'], 1.7348410253164968)],
 [(['panelSpecs', 'pv', 'tau_g'], 0.8789366862445004),
  (['panelSpecs', 'meander', 'lambd_air'], 0.0004293164670864924),
  (['hyp', 'coeff_h_top_forced_range0'], 1.5225394212836385),
  (['hyp', 'coeff_h_top_forced_range1'], 1.554462536334269),
  (['hyp', 'coeff_h_top_forced_range2'], 1.7399444856444166)],
 [(['panelSpecs', 'pv', 'tau_g'], 0.8789366891464921),
  (['panelSpecs', '

In [16]:
df_res[['Qdot_top_conv','Qdot_top_rad','Qdot_PV_sky','Qdot_absfin_back_conv','Qdot_absfin_back_rad','Qdot_tube_back_conv', 'Qdot_tube_back_rad']]

,Qdot_top_conv,Qdot_top_rad,Qdot_PV_sky,Qdot_absfin_back_conv,Qdot_absfin_back_rad,Qdot_tube_back_conv,Qdot_tube_back_rad
0,84.598048,119.266615,121.947328,6.561863,10.345963,-1.071539,-0.286517
1,84.749465,119.382967,122.067873,6.598211,10.392210,-1.036177,-0.265746
2,143.685527,114.621730,118.426076,6.885738,10.748643,-0.163988,0.217420
3,145.175691,115.613398,119.467214,7.111350,11.052281,0.075194,0.351709
4,228.418015,106.035123,111.376159,6.904298,10.756583,0.661812,0.661664
5,227.180859,105.538520,110.844939,6.786183,10.595530,0.531768,0.589304
6,117.964479,171.809215,176.570778,15.768394,22.884253,9.051309,5.195438
7,118.153043,172.010517,176.779811,15.819003,22.945726,9.110185,5.222129
8,199.913886,164.544615,170.910208,16.155864,23.329164,10.379058,5.784437
9,198.712452,163.768479,170.093773,15.960913,23.089507,10.141434,5.678775


In [ ]:
df_res['T_absfin_mean']

In [21]:
df_res[['T_fluid_mean','T_absfin_mean','T_ins_absfin_mean']] - 273.15

,T_fluid_mean,T_absfin_mean,T_ins_absfin_mean
0,26.054701,39.924736,37.606808
1,26.002624,39.865459,37.534073
2,25.780281,38.055785,35.540011
3,25.764251,37.971856,35.368981
4,25.548815,35.905591,33.291699
5,25.538799,35.935270,33.367523
6,34.129457,45.022459,39.144762
7,34.116588,44.997672,39.101958
8,33.887512,43.108703,37.015810
9,33.900427,43.174073,37.151288


In [ ]:
for subkey in mapping_parameters_bounds['panelSpecs'][key]:

In [ ]:
panelSpecs_copy['part1']['alpha_g']

In [ ]:
params_minima

In [ ]:
df_res = simulation_function([param[1] for param in params_minima[0]], panelSpecs_copy, hyp_copy, steadyStateConditions_df,mapping_parameters_bounds)

In [ ]:
steadyStateConditions_df['model'] = df_res['Qdot_tube_fluid']
steadyStateConditions_df['relative_error'] = (((steadyStateConditions_df['Qdot'] - steadyStateConditions_df['model']) / steadyStateConditions_df['Qdot'])*100)
steadyStateConditions_df[['Qdot', 'model', 'T_m - T_amb', 'u', 'relative_error']]

In [ ]:
params_minima

In [ ]:
print(panelSpecs['part2']['tau_g'])
print(panelSpecs['part2']['alpha_g'])
print(panelSpecs['part2']['alpha_PV'])
print(panelSpecs['part2']['eps_PV'])
# print(panelSpecs['part2']['eps_g'])

In [ ]:
params_minima

In [ ]:
[param[1] for param in params_minima[0]]

In [ ]:
df_res['Qdot_tube_fluid']

In [11]:
flatten_params_minima = list(chain(*params_minima))

In [ ]:
for params in tqdm(params_minima, desc="Simulations"):
    df_res_minima.append( simulation_function(params, panelSpecs_copy, hyp_copy, steadyStateConditions_df) )
    rmse_minima.append( np.sqrt(((df_res['Qdot_tube_fluid'] - steadyStateConditions_df['Qdot']) ** 2).mean()) )

In [25]:
min_params = [ results[0]['x'][i] for i in range(len(results[0]['x']))]

In [27]:
df_res = simulation_function(min_params, panelSpecs_copy, hyp_copy, steadyStateConditions_df)

In [31]:
steadyStateConditions_df['model'] = df_res['Qdot_tube_fluid']

In [ ]:
steadyStateConditions_df

In [ ]:
df_res = simulation_function([[0.6]], panelSpecs, hyp, steadyStateConditions_df)

In [30]:
# Create the initial guess as the midpoint of the bounds
initial_guess = [
    (b[0] + b[1]) / 2 for b in bounds
]

# Prepare the templates (copies of your original dictionaries)
panelSpecs_template = panelSpecs.copy()
hyp_copy = hyp.copy()

In [28]:
initial_guess = [0.2]

In [31]:
# Perform the optimization using L-BFGS-B method
result = sco.minimize(
    objective_function,
    x0=initial_guess,
    args=(panelSpecs_copy, hyp_copy, steadyStateConditions_df, steadyStateConditions_df),
    bounds=bounds,
    method='L-BFGS-B'
)

# Extract the optimal parameters
optimal_params = result.x

In [31]:
# Define bounds as a list of tuples
bounds_list = bounds

result = sco.differential_evolution(
    objective_function,
    bounds=bounds_list,
    args=(panelSpecs_template, hyp_copy, steadyStateConditions_df, steadyStateConditions_df),
    strategy='best1bin',
    maxiter=1000,
    popsize=15,
    tol=0.01,
)

optimal_params = result.x

In [ ]:
optimal_params

In [11]:
df_res, list_res = simulation_function(params, panelSpecs, hyp, steadyStateConditions_df)

In [ ]:
df_res_list = []
mae_list = []
rmse_list = []

for i, params in tqdm(enumerate(params_list), total=len(params_list)):

    df_res = simulation_function(params, panelSpecs, hyp, steadyStateConditions_df)
    mae = np.abs(df_res['Qdot_tube_fluid'] - steadyStateConditions_df['Qdot']).mean()
    rmse = np.sqrt(((df_res['Qdot_tube_fluid'] - steadyStateConditions_df['Qdot']) ** 2).mean())

    df_res_list.append(df_res)
    mae_list.append(mae)
    rmse_list.append(rmse)

In [ ]:
min_index = rmse_list.index(min(rmse_list))
params_list[min_index]

In [33]:
params_list = np.array(list(product(np.arange(0.80,0.98,0.02),
                                        np.array([0.05]),
                                        np.array([0.95]),
                                        np.array([0.9]),
                                        np.array([0.9]),
                                        np.array([0.5e-3,1e-3,1.5e-3]),
                                        np.array([0.5e-3,1e-3,1.5e-3])
                                        )))

In [22]:
params_list = np.array(list(product(np.arange(0.1,0.98,0.02)
                                        ))).tolist()

In [ ]:
params_list

In [39]:
params = ranges_matrix.tolist()

In [ ]:
len(params)

In [16]:
steadyStateConditions_df['Qdot/AG'] = steadyStateConditions_df['Qdot'] / panelSpecs['AG']

In [17]:
dic_names = {'T_in' : 'T_fluid_in0',
             'T_out' : 'T_fluid_out',
             'Qdot_1_AG' : 'Qdot/AG',
             'T_amb' : 'T_amb',
             'u' : 'u'}

In [21]:
tab_mat,tab_res,X = nm.find_SK(steadyStateConditions_df, dic_names, columns_list=[1,1,1,1,0,0,1,0,1])

In [ ]:
nm.display_a_i(X)

In [ ]:
steadyStateConditions_df

In [23]:
steadyStateConditions_df[f'{dic_names["Qdot_1_AG"]}_model'] = tab_mat @ X[0]

In [25]:
steadyStateConditions_df['relative error'] = (steadyStateConditions_df[f'{dic_names["Qdot_1_AG"]}_model'] - steadyStateConditions_df[dic_names['Qdot_1_AG']]) / steadyStateConditions_df[dic_names['Qdot_1_AG']]

In [ ]:
u_range = np.arange(0,2.8,0.01)
h_top_forced_array = np.array([bht.h_top_forced(25+273.15, 35+273.15, u, 1.8) for u in u_range])
h_top_forced_turb_array = np.array([bht.h_top_forced_turbulent(25+273.15, 35+273.15, u, 1.8) for u in u_range])

In [ ]:
h_df = pd.DataFrame({'u' : u_range, 'h_top_forced' : h_top_forced_array, 'h_top_forced_turb' : h_top_forced_turb_array})
h_df['ratio'] = h_df['h_top_forced_turb'] / h_df['h_top_forced']
# sns.histplot(h_df['ratio'], bins = 20)

In [ ]:
# plt.plot(u_range,h_top_forced_array,label='laminar then turbulent')
# plt.plot(u_range,h_top_forced_turb_array,label='turbulent')
plt.plot(u_range,h_df['ratio'])

In [11]:
# df_one,res = ty.simu_one_steady_state_all_he(panelSpecs,sC,hyp)
# df_residuals = ty.recap_residuals(panelSpecs, df_one, res)

In [13]:
df_res['Qdot_exp'] = steadyStateConditions_df['Qdot']
df_res['error'] = df_res['Qdot_tube_fluid'] - df_res['Qdot_exp']

In [ ]:
df = df_res.copy()

df = df.loc[df['u'] == 2.7]

ht_signs = {'Qdot_sun_PV': 1, 'Qdot_sun_glass': 1, 'Qdot_top_conv': -1, 'Qdot_top_rad': -1, 'Qdot_PV_sky': -1, 'Qdot_absfin_back_conv': -1, 'Qdot_absfin_back_rad': -1, 'Qdot_Base_back': -1, 'Qdot_tube_back_conv': -1, 'Qdot_tube_back_rad': -1, 'Qdot_tube_sky': -1}
ht_labels = ['Qdot_sun_PV', 'Qdot_sun_glass', 'Qdot_top_conv', 'Qdot_top_rad', 'Qdot_PV_sky', 'Qdot_absfin_back_conv', 'Qdot_absfin_back_rad', 'Qdot_tube_back_conv', 'Qdot_tube_back_rad', 'Qdot_tube_sky']
colors_ht = [RD_config.get_hex_code('yellow',100), RD_config.get_hex_code('orange',100),RD_config.get_hex_code('green',150), RD_config.get_hex_code('blue',150),RD_config.get_hex_code('blue',100),RD_config.get_hex_code('green',100),RD_config.get_hex_code('grey',100),RD_config.get_hex_code('green',50),RD_config.get_hex_code('grey',50),RD_config.get_hex_code('blue',50)]

# Calculate the total positive heat transfers for each T_m - T_amb
df['positive_total'] = df[[label for label in ht_labels if ht_signs[label] == 1]].sum(axis=1)

# Initialize the figure
fig = go.Figure()

# Define the bar width
bar_width = 0.4

# Add traces for positive heat transfers (gains)
for i, label in enumerate(ht_labels):
    if ht_signs[label] == 1:
        fig.add_trace(go.Bar(
            x=df['T_m - T_amb'],
            y=df[label],
            name=label,
            width=[bar_width] * len(df),
            hovertext=round(df[label],0),
            hoverinfo='text',
            marker=dict(color=colors_ht[i])
        ))

# Add traces for negative heat transfers (losses) with base starting from the positive total
label_sum = []
for i, label in enumerate(ht_labels):
    if ht_signs[label] == -1:
        base = df['positive_total'] - df[[lbl for lbl in ht_labels if (ht_signs[lbl] == -1) & (lbl not in label_sum)]].sum(axis=1)
        fig.add_trace(go.Bar(
            x=df['T_m - T_amb'] + bar_width,  # Offset for side-by-side display
            y=df[label],
            name=label,
            width=[bar_width] * len(df),
            base=base,
            hovertext=round(df[label],0),
            hoverinfo='text',
            marker=dict(color=colors_ht[i])
        ))
        label_sum.append(label)


fig.add_trace(go.Scatter(
    x=df['T_m - T_amb'],
    y=df['Qdot_exp'],
    mode='lines',
    name='Qdot exp',
    line=dict(color='black', width=2)
))

fig.add_trace(go.Scatter(
    x=df['T_m - T_amb'],
    y=df['Qdot_tube_fluid'],
    mode='lines',
    name='Qdot exp',
    line=dict(color='grey', width=2)
))

# Update layout for stacked bars
fig.update_layout(barmode='relative', yaxis_title='Heat Transfer', xaxis_title='T_m - T_amb')


# Show the figure
fig.show()

In [ ]:
df['T_m - T_amb']

In [ ]:
df['negative_total']

In [ ]:
# df_res[['Qdot_tube_fluid','Qdot_exp','error']]

In [ ]:
mae = np.abs(df_res['error']).mean()
print(mae)

In [ ]:
wm = 7

# Deuxième écriture plus simple pour modifier les coefficients
mapping_matrix = np.array([
                            wm*[''],
                            ['tau_g', 'alpha_g', 'alpha_PV', 'eps_PV', 'eps_g', 'l_c', 'lambd_air'],
                           ])

part_mapping_matrix = [
                        np.nan,
                        [['part1','part2','part3'] for _ in range(wm)]
                        ]

ranges_matrix = np.array(list(product(np.arange(0.80,0.98,0.02),
                                        np.array([0.05]),
                                        np.array([0.95]),
                                        np.array([0.9]),
                                        np.array([0.9]),
                                        np.array([0.5e-3,1e-3,1.5e-3]),
                                        np.array([0.5e-3,1e-3,1.5e-3])
                                        )))

In [ ]:
panelSpecs['decomp']

In [ ]:
panelSpecs['part1']

In [24]:
wm = 1

# Deuxième écriture plus simple pour modifier les coefficients
mapping_matrix = np.array([
                            wm*[''],
                            ['lambd_air'],
                           ])

part_mapping_matrix = [
                        np.nan,
                        [['part1','part2','part3'] for _ in range(wm)]
                        ]

ranges_matrix = np.array(list(product(
                                        np.array([0.1e-3,10e-3,1.1e-4])
                                        )))

In [ ]:
ranges_matrix

In [ ]:
part_mapping_matrix[1]

In [ ]:
panelSpecs.keys()

In [ ]:
flattened_parameters

In [ ]:
df_res_list[9][['T_amb','u','T_fluid_in','Qdot_tube_fluid','Qdot exp','error']]

In [ ]:
min_index = mae_list.index(min(mae_list))
print(min_index)

In [ ]:
min(mae_list)

In [ ]:
df_res_list[3][['Qdot_tube_fluid','Qdot exp','error']]

In [ ]:
mae_list

In [ ]:
plt.scatter(np.arange(0,len(rmse_list),1),rmse_list)

In [ ]:
panelSpecs['pv']['tau_g']

In [ ]:
len(ranges_matrix)*2/3600

In [ ]:
all_combinations = list(product(range1, range2, range3))
matrix = np.array(all_combinations)

In [ ]:
matrix

Calibrage modèle multi-design

In [ ]:
# Permet de modifier une valeur du dictionnaire hyp
## Par exemple init_hyp('method_h_back_abs','free') va retourner le dictionnaire hyp avec la valeur 'method_h_back_abs' modifié en 'free'
def init_hyp(coeff, method) : 
    hyp[coeff] = method
    return(hyp)

Création de la matrice du modèle

In [ ]:
# Récupération des résultats expérimentaux

X_list_df = pd.read_excel(exp_path,header = 2) ## On ne prend pas en compe les 2 premières lignes // EN FONCTION DE COMMENT SONT PR2SENT2S LES RES
X_list = X_list_df.to_dict(orient='records')

In [ ]:
# Création de la matrice "A" sous forme de dictionnaire

measured_case = 0 # nb_measured_case = len(X_tot)
case = 0 # nb_case = len(stepConditions) il faudrait faire correspondre les lignes de stepConditions avec le numéro des measuredcase comme ça on a qu'une variable

# Dictionnaire des paramètres du modèle commun à tous les designs "alpha_0"
basic_parameters = dict([
    ('basic_parameters_1',1),
    ('basic_parameters_2',1),
    ('basic_parameters_3',1),
    ('basic_parameters_4',1),
    ('basic_parameters_5',1),
    ('basic_parameters_6',1),
    ('basic_parameters_7',1),
    ('basic_parameters_8',1),
    ('basic_parameters_9',1),
    ('basic_parameters_10',1),
    ])

# Matrice du modèle "A" 
model_matrix = dict(['alpha_0',basic_parameters])

# Plutôt faire l'inverse et créer le dictionnaire à partir d'une DataFrame puis sortir les différents X
# for i in range(nb_measured_case) :
# Point de données
X = dict([('componentSpecs',componentSpecs), ('stepConditions',stepConditions[case])]) 
# X_tot = dict([ 
#    ('X_{measured_case}',X)
#   ])

# Dictionnaire des paramètres du modèle liés à la géométrie "alpha_j"
design_parameters = dict([
    ('tau_g_{measured_case}', X['componentSpecs']['pv']['tau_g']), # Transmittance glass
    ('alpha_g_{measured_case}', X['componentSpecs']['pv']['alpha_g']), # Absorptance glass
    ('eps_PV_{measured_case}', X['componentSpecs']['pv']['eps_PV']), # Émissivité cellule PV
    ('eps_g_{measured_case}', X['componentSpecs']['pv']['eps_g']), # Émissivité glass
    ('eps_hx_top_{measured_case}', X['componentSpecs']['main']['eps_hx_top']), # Émissivité de l'échangeur thermique sur le panneau
    ('eps_hx_back_{measured_case}', X['componentSpecs']['main']['eps_hx_back']), # Émissivité de l'échangeur thermique à l’arrière du panneau
    ('eps_ins_{measured_case}', X['componentSpecs']['main']['eps_ins']), # Émissivité isolant
    ('lambd_air_{measured_case}',  X['componentSpecs']['main']['lambd_air']) # Épaisseur de la couche d'air
    ])

model_matrix['alpha_{measured_case}'] = design_parameters
#fin boucle for

In [ ]:
# Création de la matrice "A" sous forme de listes

measured_case = 0 # nb_measured_case = len(X_list)
case = 0 # nb_case = len(stepConditions) il faudrait faire correspondre les lignes de stepConditions avec le numéro des measuredcase comme ça on a qu'une variable

# Matrice du modèle "A" 
model_matrix = np.ones(10)

# Plutôt faire l'inverse et créer le dictionnaire à partir d'une DataFrame puis sortir les différents X
# for i in range(nb_measured_case) :
# Point de données
X = dict([('componentSpecs',componentSpecs), ('stepConditions',stepConditions[case])]) 
# X_list = dict([ 
#    ('X_{measured_case}',X)
#   ])

# Dictionnaire des paramètres du modèle liés à la géométrie "alpha_j"
design_parameters = np.zeros(10)
design_parameters[0] = X['componentSpecs']['pv']['tau_g'] # Transmittance glass
design_parameters[1] =  X['componentSpecs']['pv']['alpha_g'] # Absorptance glass
design_parameters[2] = X['componentSpecs']['pv']['eps_PV'] # Émissivité cellule PV
design_parameters[3] = X['componentSpecs']['pv']['eps_g'] # Émissivité glass
design_parameters[4] = X['componentSpecs']['main']['eps_hx_top'] # Émissivité de l'échangeur thermique sur le panneau
design_parameters[5] = X['componentSpecs']['main']['eps_hx_back'] # Émissivité de l'échangeur thermique à l’arrière du panneau
design_parameters[6] = X['componentSpecs']['main']['eps_ins'] # Émissivité isolant
design_parameters[7] = X['componentSpecs']['main']['lambd_air'] # Épaisseur de la couche d'air


model_matrix= np.vstack((model_matrix, design_parameters))

#fin boucle for
model_matrix

In [ ]:
m = len(correspondance_matrice)
nb_measured_case = len(X_list)

# MAJ de hyp en fonction des méthodes que l'on veut
hyp = init_hyp("method_h_back_abs", "free")


def Qdot_model_tube_fluid(X_list, model_matrix, correspondance_matrice) :
        Qdot_list = []
        
        # Actualise le modèle
        for i in range(m) : 
                if correspondance_matrice[i, 0] != 0. : 
                       hyp[correspondance_matrice[i,0]] = model_matrix[i, 0]

        # Actualise la géométrie
        for j in range(nb_measured_case) : 
                for i in range(m) :
                        if correspondance_matrice[i, 0] != 0. :
                                X_list[j]['componentSpecs'][correspondance_matrice[i,1]] = model_matrix[i, j]
                df_one = ty.simu_one_steady_state_all_he(X_list[j]['componentSpecs'], X_list[j]['stepConditions'], hyp)[0]
                Qdot_list.append(df_one['Qdot_tube_fluid'])
        return(Qdot_list)